In [87]:
import json
import string
import random
import requests

In [88]:
# data generation
n = 100
len = 10
emails = []
usernames = []
passwords = []
validation = set()

new_emails = []
new_passwords = []
email_validation = set()
password_validation = set()

for i in range(n):
    em = ''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase + string.digits, k=len)) + '@gmail.com'
    user = ''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase + string.digits, k=len))
    pwd = ''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase + string.digits, k=len))
    
    if i % 2 == 0:
        new_em = ''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase + string.digits, k=len)) + '@gmail.com'
        new_emails.append(new_em)
        email_validation.add((user, new_em, pwd))
    else:
        new_pwd = ''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase + string.digits, k=len))
        new_passwords.append(new_pwd)
        password_validation.add((user, em, new_pwd))
    
    emails.append(em)
    usernames.append(user)
    passwords.append(pwd)
    validation.add((user, em, pwd))    

In [89]:
# print(emails[:10])
# print(usernames[:10])
# print(passwords[:10])

In [90]:
import os
import dotenv

dotenv_path = os.path.join(os.path.abspath(''), '../', '.env')

dotenv.load_dotenv(dotenv_path)

node_url = os.getenv("EXPO_NODE_URL") + ':' + os.getenv("PORT")
flask_url = os.getenv("EXPO_FLASK_URL")

user_url = node_url + "/api/v1/user"
flask_url = flask_url + ":9000"

In [91]:
# Register API Calls
success = 0
for i in range(n):
    data = {'username': usernames[i], 'email': emails[i], 'password':  passwords[i]}
    r = requests.post(user_url + "/register", json=data)
    if r.status_code == 201:
        success += 1

print('Testing creating new accounts')
print('-----------------------------')
print('Expected Success Rate: 100%')
print(f'{success / n * 100:.2f}% of registrations were successful')

Testing creating new accounts
-----------------------------
Expected Success Rate: 100%
100.00% of registrations were successful


In [92]:
# Register Validation
valid = 0
rows = requests.get(user_url + "/")
for r in rows.json():
    if (r['username'], r['email'], r['password']) in validation:
        valid += 1
        
print('Validating account creation')
print('---------------------------')
print(f'{valid} accounts out of {n} were found in the database')

Validating account creation
---------------------------
100 accounts out of 100 were found in the database


In [93]:
# Register with existing emails
fail = 0
for i in range(n):
    data = {'username': usernames[i], 'email': emails[i], 'password':  passwords[i]}
    r = requests.post(user_url + "/register", json=data)
    if r.status_code == 500:
        fail += 1
        
print('Testing registering with existing email')
print('---------------------------------------')
print('Expected Fail Rate: 100%')
print(f'{fail / n * 100:.2f}% of registrations failed')

Testing registering with existing email
---------------------------------------
Expected Fail Rate: 100%
100.00% of registrations failed


In [94]:
# Register with missing field(s)
em = 0
user = 0
pwd = 0
for i in range(n):
    data = {'username': usernames[i], 'email': emails[i], 'password':  passwords[i]}
    if i % 3 == 0:
        data['username'] = ''
    if i % 3 == 1:
        data['email'] = ''
    if i % 3 == 2:
        data['password'] = ''
        
    r = requests.post(user_url + "/register", json=data)
    if r.status_code == 400:
        if r.json()['message'] == 'username required':
            user += 1
        if r.json()['message'] == 'email required':
            em += 1
        if r.json()['message'] == 'password required':
            pwd += 1
            
print('Testing registering with missing fields')
print('---------------------------------------')
print('Expected Fail Rate: 100%')
print(f'{(user + em + pwd) / n * 100:.2f}% of registrations failed')
print(f'Missing usernames: {user}')
print(f'Missing emails: {em}')
print(f'Missing passwords: {pwd}')

Testing registering with missing fields
---------------------------------------
Expected Fail Rate: 100%
100.00% of registrations failed
Missing usernames: 34
Missing emails: 33
Missing passwords: 33


In [95]:
# Login API Calls

In [96]:
# Login Validation

In [97]:
# Preference Update API Calls

In [98]:
# Preference Update Validation

In [99]:
# Reset Email API Calls
success = 0
for i in range(0, n, 2):
    data = {'oldEmail': emails[i], 'newEmail': new_emails[i//2], 'password':  passwords[i]}
    r = requests.post(user_url + "/resetEmail", json=data)
    if r.status_code == 201:
        success += 1

print('Testing resetting email')
print('-----------------------')
print('Expected Success Rate: 100%')
print(f'{success / (n//2) * 100:.2f}% of resets were successful')

Testing resetting email
-----------------------
Expected Success Rate: 100%
100.00% of resets were successful


In [100]:
# Reset Email Validation
valid = 0
rows = requests.get(user_url + "/")
for r in rows.json():
    if (r['username'], r['email'], r['password']) in email_validation:
        valid += 1
        
print('Validating email reset')
print('----------------------')
print(f'{valid} accounts out of {n//2} were found in the database')

Validating email reset
----------------------
50 accounts out of 50 were found in the database


In [101]:
# Reset Email with non-existing user
fail = 0
for i in range(0, n, 2):
    data = {'oldEmail': emails[i], 'newEmail': new_emails[i//2], 'password':  passwords[i]}
    r = requests.post(user_url + "/resetEmail", json=data)
    if r.status_code == 500 and r.json()['message'] == 'user does not exist':
        fail += 1

print('Testing resetting email with non-existing user')
print('----------------------------------------------')
print('Expected Fail Rate: 100%')
print(f'{fail / (n//2) * 100:.2f}% of resets failed')

Testing resetting email with non-existing user
----------------------------------------------
Expected Fail Rate: 100%
100.00% of resets failed


In [102]:
# Reset Email with missing field(s)
fail = 0
for i in range(0, n, 2):
    data = {'oldEmail': emails[i], 'newEmail': new_emails[i//2], 'password':  passwords[i]}
    if i % 3 == 0:
        data['oldEmail'] = ''
    if i % 3 == 1:
        data['newEmail'] = ''
    if i % 3 == 2:
        data['password'] = ''
        
    r = requests.post(user_url + "/resetEmail", json=data)
    if r.status_code == 500 and r.json()['message'] == 'Please provide username or password':
        fail += 1

print('Testing resetting email with missing field(s)')
print('---------------------------------------------')
print('Expected Fail Rate: 100%')
print(f'{fail / (n//2) * 100:.2f}% of resets failed')

Testing resetting email with missing field(s)
---------------------------------------------
Expected Fail Rate: 100%
100.00% of resets failed


In [103]:
# Reset Password API Calls
success = 0
for i in range(1, n, 2):
    data = {'oldPassword': passwords[i], 'email': emails[i], 'newPassword':  new_passwords[i//2]}
    r = requests.post(user_url + "/resetPassword", json=data)
    if r.status_code == 201:
        success += 1

print('Testing resetting password')
print('--------------------------')
print('Expected Success Rate: 100%')
print(f'{success / (n//2) * 100:.2f}% of resets were successful')

Testing resetting password
--------------------------
Expected Success Rate: 100%
100.00% of resets were successful


In [104]:
# Reset Password Validation
valid = 0
rows = requests.get(user_url + "/")
for r in rows.json():
    if (r['username'], r['email'], r['password']) in email_validation:
        valid += 1
        
print('Validating password reset')
print('-------------------------')
print(f'{valid} accounts out of {n//2} were found in the database')

Validating password reset
-------------------------
50 accounts out of 50 were found in the database


In [105]:
# Reset Password with non-existing user
fail = 0
for i in range(1, n, 2):
    data = {'oldPassword': passwords[i], 'email': emails[i], 'newPassword':  new_passwords[i//2]}
    r = requests.post(user_url + "/resetPassword", json=data)
    if r.status_code == 500 and r.json()['message'] == 'user does not exist':
        fail += 1

print('Testing resetting password with non-existing user')
print('-------------------------------------------------')
print('Expected Fail Rate: 100%')
print(f'{fail / (n//2) * 100:.2f}% of resets failed')

Testing resetting password with non-existing user
-------------------------------------------------
Expected Fail Rate: 100%
100.00% of resets failed


In [106]:
# Reset Password with missing field(s)
fail = 0
for i in range(0, n, 2):
    data = {'oldPassword': passwords[i], 'email': emails[i], 'newPassword':  new_passwords[i//2]}
    if i % 3 == 0:
        data['oldPassword'] = ''
    if i % 3 == 1:
        data['email'] = ''
    if i % 3 == 2:
        data['newPassword'] = ''
        
    r = requests.post(user_url + "/resetPassword", json=data)
    if r.status_code == 500 and r.json()['message'] == 'Please provide username or password':
        fail += 1

print('Testing resetting password with missing field(s)')
print('------------------------------------------------')
print('Expected Fail Rate: 100%')
print(f'{fail / (n//2) * 100:.2f}% of resets failed')

Testing resetting password with missing field(s)
------------------------------------------------
Expected Fail Rate: 100%
100.00% of resets failed


In [107]:
# Delete Users